# Predicting NYC Taxi Fares with XGBoost + Ray Train
## DATS 6450 — Big Data & Cloud Computing

**Prerequisites**: Complete `tutorial.ipynb` first — this notebook assumes Ray, the NYC Taxi dataset, and your S3 bucket are already familiar.

**What you will build**: A gradient-boosted tree model that predicts taxi `fare_amount` from trip features — trained in parallel using **Ray Train**.

**Environment**: t3.large EC2 (2 vCPUs, 8 GB RAM) with `LabInstanceProfile` attached.

## What You Will Learn

| Section | Topic |
|---------|-------|
| 1 | Environment setup & imports |
| 2 | Data loading & feature engineering with Ray Data |
| 3 | Exploratory analysis — what drives fares? |
| 4 | Baseline model — serial XGBoost on a single node |
| 5 | Distributed training with **Ray Train + XGBoost** |
| 6 | Hyperparameter tuning with **Ray Tune** |
| 7 | Model evaluation — errors, feature importance, residuals |
| 8 | Saving the model to S3 & batch prediction |
| 9 | Cleanup & summary |

### Why XGBoost for fare prediction?

- **Tabular data** — tree models consistently outperform neural networks on structured data like this
- **Mixed feature types** — XGBoost handles integers (hour, zones), floats (distance), and categoricals naturally
- **Interpretable** — feature importance tells us which inputs matter most
- **Ray Train integration** — `xgboost-ray` lets the same XGBoost model train across multiple Ray workers with zero code changes to the model itself

### Target variable

We predict **`fare_amount`** — the metered fare set by the TLC rate formula. We intentionally exclude `tip_amount`, `tolls_amount`, and `total_amount` since those are downstream of the fare.

---
## Section 1 — Environment Setup

In [ ]:
!uv sync

In [ ]:
import ray
import ray.train
from ray.train.xgboost import XGBoostTrainer
from ray.train import ScalingConfig, RunConfig, CheckpointConfig
import ray.tune as tune
from ray.tune.schedulers import ASHAScheduler

import xgboost as xgb
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.fs as pafs
import pyarrow.parquet as pq
import boto3
import matplotlib.pyplot as plt
import seaborn as sns
import time
import io
import os
import pickle

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

print(f"Ray:        {ray.__version__}")
print(f"XGBoost:    {xgb.__version__}")
print(f"pandas:     {pd.__version__}")
print(f"scikit-learn: imported OK")

In [ ]:
# ✏️  FILL IN YOUR BUCKET NAME (same one used in tutorial.ipynb)
YOUR_BUCKET_NAME = "YOUR_BUCKET_NAME_HERE"

assert YOUR_BUCKET_NAME != "YOUR_BUCKET_NAME_HERE", \
    "Please replace YOUR_BUCKET_NAME_HERE with your actual bucket name!"

MODEL_S3_PREFIX = f"s3://{YOUR_BUCKET_NAME}/ray-tutorial/models/"
print(f"Models will be saved to: {MODEL_S3_PREFIX}")

In [ ]:
ray.init(ignore_reinit_error=True)
print(f"Ray initialized. Resources: {ray.available_resources()}")

---
## Section 2 — Data Loading & Feature Engineering

We reuse the same cleaning and feature-engineering logic from `tutorial.ipynb`, applied here with Ray Data across **3 months** of NYC Taxi data (Jan–Mar 2023, ~9M raw rows).

### Feature design

| Feature | Type | Rationale |
|---------|------|----------|
| `trip_distance` | float | Primary fare driver (metered by distance) |
| `trip_duration_min` | float | Secondary driver (metered by time in traffic) |
| `pickup_hour` | int | Rush-hour surcharges, demand pricing |
| `pickup_dow` | int | Weekday vs. weekend patterns |
| `PULocationID` | int | Origin zone — airport trips have flat fares |
| `DOLocationID` | int | Destination zone |
| `passenger_count` | int | Minor effect; kept for completeness |
| `RatecodeID` | int | Critical: 1=standard, 2=JFK flat ($70), 3=Newark, 5=negotiated |

> **Key insight**: `RatecodeID` is the single most predictive feature — JFK flat-rate trips will be ~$70 regardless of distance. The model should learn this quickly.

In [ ]:
s3_anon = pafs.S3FileSystem(anonymous=True, region="us-east-1")

MONTHS = ["2023-01", "2023-02", "2023-03"]
PARQUET_PATHS = [
    f"nyc-tlc/trip data/yellow_tripdata_{m}.parquet"
    for m in MONTHS
]

# Feature columns we want to keep (plus the target)
FEATURE_COLS = [
    "trip_distance",
    "PULocationID",
    "DOLocationID",
    "passenger_count",
    "RatecodeID",
]
ENGINEERED_COLS = ["trip_duration_min", "pickup_hour", "pickup_dow"]
TARGET = "fare_amount"

ALL_NEEDED = FEATURE_COLS + ENGINEERED_COLS + [TARGET,
    "tpep_pickup_datetime", "tpep_dropoff_datetime"]  # needed to engineer features

print(f"Loading {len(MONTHS)} months of NYC Taxi data from S3...")
print(f"Paths: {[p.split('/')[-1] for p in PARQUET_PATHS]}")

In [ ]:
def prepare_for_model(df: pd.DataFrame) -> pd.DataFrame:
    """
    Combined clean + feature-engineer + select pass.
    Runs in parallel on each Ray Data partition.
    """
    # --- Clean timestamps ---
    for col in ["tpep_pickup_datetime", "tpep_dropoff_datetime"]:
        if col in df.columns and hasattr(df[col], "dt"):
            if df[col].dt.tz is not None:
                df[col] = df[col].dt.tz_localize(None)

    # --- Filter outliers ---
    df = df[(df["fare_amount"] >= 2.50) & (df["fare_amount"] <= 300)]
    df = df[(df["trip_distance"] > 0) & (df["trip_distance"] <= 60)]
    df = df[(df["passenger_count"] >= 1) & (df["passenger_count"] <= 6)]
    df = df.dropna(subset=["tpep_pickup_datetime", "tpep_dropoff_datetime",
                            "RatecodeID", "PULocationID", "DOLocationID"])
    df = df[df["tpep_dropoff_datetime"] > df["tpep_pickup_datetime"]]

    # --- Engineered features ---
    df["trip_duration_min"] = (
        (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"])
        .dt.total_seconds() / 60
    )
    df = df[(df["trip_duration_min"] >= 1) & (df["trip_duration_min"] <= 180)]

    df["pickup_hour"] = df["tpep_pickup_datetime"].dt.hour
    df["pickup_dow"]  = df["tpep_pickup_datetime"].dt.dayofweek

    # --- Cast types for XGBoost ---
    int_cols = ["PULocationID", "DOLocationID", "passenger_count",
                "RatecodeID", "pickup_hour", "pickup_dow"]
    for c in int_cols:
        df[c] = df[c].astype(int)

    # --- Select only the columns the model needs ---
    keep = FEATURE_COLS + ENGINEERED_COLS + [TARGET]
    return df[keep]


print("Building Ray Data pipeline...")
t0 = time.time()

raw_ds = ray.data.read_parquet(PARQUET_PATHS, filesystem=s3_anon)
model_ds = raw_ds.map_batches(prepare_for_model, batch_format="pandas")

# Materialize — triggers all the reading and transforms
print("Materializing dataset (reading S3 + applying transforms)...")
print("This takes ~2–4 min on t3.large for 3 months.")
model_df = model_ds.to_pandas()

elapsed = time.time() - t0
print(f"\nDone in {elapsed:.0f}s")
print(f"Dataset: {len(model_df):,} rows × {len(model_df.columns)} columns")
print(f"Memory: {model_df.memory_usage(deep=True).sum() / 1e6:.0f} MB")
print(f"\nColumns: {list(model_df.columns)}")

In [ ]:
print("Dataset summary:")
print(model_df.describe().round(2))

---
## Section 3 — Exploratory Analysis: What Drives Fares?

Before training, let's look at the relationships between features and `fare_amount`. This guides feature selection and gives us a sanity check that the data looks right.

In [ ]:
# Work on a 50k sample for fast plotting
sample = model_df.sample(50_000, random_state=42)

fig, axes = plt.subplots(2, 3, figsize=(16, 9))
fig.suptitle("NYC Taxi — Feature Relationships with Fare Amount", fontsize=13)

# 1. Distance vs. Fare (the primary relationship)
ax = axes[0, 0]
ax.hexbin(sample["trip_distance"], sample["fare_amount"],
          gridsize=50, cmap="Blues", mincnt=1)
ax.set_xlabel("Trip Distance (miles)")
ax.set_ylabel("Fare Amount ($)")
ax.set_title("Distance vs. Fare")
ax.set_xlim(0, 30)
ax.set_ylim(0, 100)

# 2. Duration vs. Fare
ax = axes[0, 1]
ax.hexbin(sample["trip_duration_min"], sample["fare_amount"],
          gridsize=50, cmap="Greens", mincnt=1)
ax.set_xlabel("Trip Duration (min)")
ax.set_ylabel("Fare Amount ($)")
ax.set_title("Duration vs. Fare")
ax.set_xlim(0, 60)
ax.set_ylim(0, 100)

# 3. Fare distribution by RatecodeID
ax = axes[0, 2]
ratecode_labels = {1: "Standard", 2: "JFK", 3: "Newark", 4: "Nassau", 5: "Negotiated", 6: "Group"}
for rc in sorted(sample["RatecodeID"].unique()):
    if rc in ratecode_labels:
        subset = sample[sample["RatecodeID"] == rc]["fare_amount"]
        ax.hist(subset.clip(0, 120), bins=40, alpha=0.5,
                label=f"{rc}: {ratecode_labels.get(rc, rc)}", density=True)
ax.set_xlabel("Fare Amount ($)")
ax.set_ylabel("Density")
ax.set_title("Fare Distribution by Rate Code")
ax.legend(fontsize=7)

# 4. Avg fare by pickup hour
ax = axes[1, 0]
hourly = model_df.groupby("pickup_hour")["fare_amount"].mean()
ax.plot(hourly.index, hourly.values, color="darkorange", marker="o", markersize=4)
ax.set_xlabel("Pickup Hour")
ax.set_ylabel("Avg Fare ($)")
ax.set_title("Avg Fare by Hour")
ax.set_xticks(range(0, 24, 3))
ax.grid(True, alpha=0.3)

# 5. Avg fare by day of week
ax = axes[1, 1]
dow_labels = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
daily = model_df.groupby("pickup_dow")["fare_amount"].mean()
ax.bar(dow_labels, daily.values, color="steelblue", alpha=0.8)
ax.set_ylabel("Avg Fare ($)")
ax.set_title("Avg Fare by Day of Week")
ax.grid(axis="y", alpha=0.3)

# 6. Fare distribution (log scale)
ax = axes[1, 2]
ax.hist(model_df["fare_amount"].clip(0, 150), bins=80,
        color="mediumslateblue", alpha=0.8, edgecolor="none")
ax.set_xlabel("Fare Amount ($)")
ax.set_ylabel("Count")
ax.set_title("Fare Amount Distribution")
ax.set_yscale("log")
ax.axvline(model_df["fare_amount"].median(), color="red",
           linestyle="--", linewidth=1.5, label=f"Median: ${model_df['fare_amount'].median():.2f}")
ax.legend()

plt.tight_layout()
plt.savefig("fare_eda.png", dpi=120, bbox_inches="tight")
plt.show()
print("Saved: fare_eda.png")

In [ ]:
# Correlation matrix — numeric features only
corr_cols = FEATURE_COLS + ENGINEERED_COLS + [TARGET]
corr = model_df[corr_cols].corr()[[TARGET]].drop(TARGET).sort_values(TARGET, ascending=False)

print("Pearson correlation with fare_amount:")
print(corr.round(3).to_string())

print("\nKey observations:")
print("  • trip_distance has the strongest linear correlation")
print("  • trip_duration_min is also strong (correlated with distance)")
print("  • RatecodeID is moderate — but nonlinear (JFK flat rate = $70)")
print("  • pickup_hour/dow have weak linear correlation but nonlinear patterns")
print("  → XGBoost captures all nonlinear interactions automatically")

---
## Section 4 — Baseline: Serial XGBoost on a Single Node

Before using Ray Train, we establish a **serial baseline** — standard XGBoost on a 500k-row sample. This:
1. Confirms the model works before adding distributed complexity
2. Gives us a timing baseline to compare against Ray Train
3. Shows us what a reasonable MAE / RMSE looks like

### Train / validation / test split strategy

We split **temporally** rather than randomly — this avoids leakage and simulates real deployment where the model is trained on past data and evaluated on future data.

```
January 2023    →  Training set   (60%)
February 2023   →  Validation set (20%)   ← used during tuning
March 2023      →  Test set       (20%)   ← final evaluation only
```

For the serial baseline we sample 500k rows from the full dataset for speed.

In [ ]:
FEATURE_COLS_MODEL = FEATURE_COLS + ENGINEERED_COLS

# Sample 500k rows for the serial baseline (fast iteration)
baseline_df = model_df.sample(500_000, random_state=42).reset_index(drop=True)

X = baseline_df[FEATURE_COLS_MODEL]
y = baseline_df[TARGET]

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42
)

print(f"Train: {len(X_train):,} rows")
print(f"Val:   {len(X_val):,} rows")
print(f"Test:  {len(X_test):,} rows")
print(f"Features: {FEATURE_COLS_MODEL}")

In [ ]:
# Serial XGBoost baseline
baseline_params = {
    "objective": "reg:squarederror",
    "n_estimators": 200,
    "max_depth": 6,
    "learning_rate": 0.1,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "tree_method": "hist",   # fast histogram-based algorithm
    "n_jobs": -1,            # use all CPUs on the node
    "random_state": 42,
}

print("Training serial XGBoost baseline...")
t0 = time.time()

baseline_model = xgb.XGBRegressor(**baseline_params)
baseline_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=50,
)

baseline_time = time.time() - t0
print(f"\nTraining complete in {baseline_time:.1f}s")

# Evaluate on test set
y_pred_baseline = baseline_model.predict(X_test)

mae_b  = mean_absolute_error(y_test, y_pred_baseline)
rmse_b = mean_squared_error(y_test, y_pred_baseline, squared=False)
r2_b   = r2_score(y_test, y_pred_baseline)

print(f"\nBaseline test-set metrics (500k sample):")
print(f"  MAE:  ${mae_b:.2f}")
print(f"  RMSE: ${rmse_b:.2f}")
print(f"  R²:   {r2_b:.4f}")

---
## Section 5 — Distributed Training with Ray Train + XGBoost

**Ray Train** wraps XGBoost's native distributed training. Under the hood:
1. Ray Train spawns `num_workers` Ray actors
2. Each actor gets a shard of the data
3. XGBoost's distributed mode (using Rabit — a reduction ring) synchronizes gradient updates across workers
4. The result is identical to single-node XGBoost — just faster when you have many CPUs/nodes

### Key Ray Train concepts

| Concept | Description |
|---------|-------------|
| `XGBoostTrainer` | Ray Train's XGBoost integration — wraps `xgb.train()` |
| `ScalingConfig` | How many workers, CPUs per worker |
| `RunConfig` | Where to save checkpoints and logs |
| `ray.data.Dataset` | Input data — Ray Train reads from this natively |

> **On t3.large (2 CPUs)**: Ray Train with 2 workers gives similar speed to serial XGBoost with `n_jobs=-1`. The architecture benefit shows on multi-node clusters — `ScalingConfig(num_workers=10)` on a 5-node cluster would use all 10 CPUs in parallel.

In [ ]:
# Convert our pandas DataFrame back to a Ray Dataset
# Ray Train reads directly from Ray Datasets (no manual sharding needed)
print("Converting pandas DataFrame to Ray Dataset...")

train_df = model_df.sample(frac=1, random_state=42).reset_index(drop=True)  # shuffle

n_total = len(train_df)
n_train = int(n_total * 0.70)
n_val   = int(n_total * 0.15)

train_split = train_df.iloc[:n_train]
val_split   = train_df.iloc[n_train:n_train + n_val]
test_split  = train_df.iloc[n_train + n_val:]

print(f"Full dataset split:")
print(f"  Train: {len(train_split):,} rows")
print(f"  Val:   {len(val_split):,} rows")
print(f"  Test:  {len(test_split):,} rows")

# Convert to Ray Datasets
train_ray_ds = ray.data.from_pandas(train_split)
val_ray_ds   = ray.data.from_pandas(val_split)
test_ray_ds  = ray.data.from_pandas(test_split)

print(f"\nRay Dataset schema: {train_ray_ds.schema()}")

In [ ]:
# Define XGBoost parameters for Ray Train
# These are identical to what you'd pass to xgb.train() directly
xgb_params = {
    "objective": "reg:squarederror",
    "eval_metric": ["rmse", "mae"],
    "max_depth": 6,
    "eta": 0.1,                 # learning rate (called 'eta' in xgb.train API)
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "min_child_weight": 5,
    "tree_method": "hist",
    "seed": 42,
}

# ScalingConfig: how Ray distributes the training
# num_workers=2 → 2 Ray actors, each getting half the data
# On t3.large we have 2 vCPUs, so 1 CPU per worker
scaling_config = ScalingConfig(
    num_workers=2,
    use_gpu=False,
)

# RunConfig: where to save results
run_config = RunConfig(
    name="xgboost_fare_prediction",
    storage_path=os.path.expanduser("~/ray_results"),
    checkpoint_config=CheckpointConfig(
        num_to_keep=1,  # save only the best checkpoint
    ),
)

print("Ray Train config ready.")
print(f"  Workers: {scaling_config.num_workers}")
print(f"  XGBoost params: {xgb_params}")

In [ ]:
# Build the XGBoostTrainer
# label_column → column in the Ray Dataset to use as the target
# datasets → dict mapping split names to Ray Datasets
trainer = XGBoostTrainer(
    params=xgb_params,
    label_column=TARGET,
    num_boost_round=200,
    datasets={
        "train": train_ray_ds,
        "valid": val_ray_ds,
    },
    scaling_config=scaling_config,
    run_config=run_config,
)

print("Starting Ray Train XGBoost distributed training...")
print(f"Training on {len(train_split):,} rows with {scaling_config.num_workers} workers")
print("(Expect ~3–6 min on t3.large)")

t0 = time.time()
result = trainer.fit()
ray_train_time = time.time() - t0

print(f"\nTraining complete in {ray_train_time:.1f}s")
print(f"Best checkpoint: {result.checkpoint}")
print(f"\nFinal metrics:")
for k, v in result.metrics.items():
    if "rmse" in k or "mae" in k:
        print(f"  {k}: {v:.4f}")

In [ ]:
# Load the trained model from the Ray Train checkpoint
from ray.train.xgboost import XGBoostPredictor

checkpoint = result.checkpoint

# Load model for evaluation
predictor = XGBoostPredictor.from_checkpoint(checkpoint)

# Predict on test set (using the Ray Dataset)
print("Running predictions on test set with Ray Train predictor...")
test_predictions_ds = predictor.predict(test_ray_ds.drop_columns([TARGET]))
test_preds_df = test_predictions_ds.to_pandas()

# Align predictions with actuals
y_test_vals = test_split[TARGET].values
y_pred_ray  = test_preds_df["predictions"].values

mae_ray  = mean_absolute_error(y_test_vals, y_pred_ray)
rmse_ray = mean_squared_error(y_test_vals, y_pred_ray, squared=False)
r2_ray   = r2_score(y_test_vals, y_pred_ray)

print(f"\nRay Train XGBoost — test-set metrics (full {len(test_split):,}-row test set):")
print(f"  MAE:  ${mae_ray:.2f}")
print(f"  RMSE: ${rmse_ray:.2f}")
print(f"  R²:   {r2_ray:.4f}")

---
## Section 6 — Hyperparameter Tuning with Ray Tune

**Ray Tune** automates hyperparameter search. It integrates with Ray Train — each trial is a full training run, and Tune manages scheduling, early stopping, and result logging.

### ASHA Scheduler

We use **Asynchronous Successive Halving (ASHA)** — an early-stopping algorithm that:
1. Starts many trials in parallel with a short `num_boost_round`
2. Keeps only the top-performing trials and gives them more rounds
3. Eliminates poor trials early — far more efficient than grid search

### Search space

We tune the four parameters with the most impact on XGBoost fare prediction:

| Parameter | Effect |
|-----------|--------|
| `max_depth` | Tree complexity — deeper = more expressive, but overfits |
| `eta` | Learning rate — smaller = more rounds needed, but smoother |
| `subsample` | Row subsampling — reduces variance |
| `colsample_bytree` | Feature subsampling per tree — reduces correlation between trees |

> **Note**: On t3.large, we run a small search (4 trials, 100 rounds max) to keep it within ~10 minutes. On a larger instance or cluster, increase `num_samples` to 20–50.

In [ ]:
# Use a 300k-row subset for tuning (faster iteration per trial)
tune_df    = train_df.sample(300_000, random_state=0).reset_index(drop=True)
n_tune     = len(tune_df)
n_tune_tr  = int(n_tune * 0.80)

tune_train_ds = ray.data.from_pandas(tune_df.iloc[:n_tune_tr])
tune_val_ds   = ray.data.from_pandas(tune_df.iloc[n_tune_tr:])

print(f"Tuning on {n_tune_tr:,} train / {n_tune - n_tune_tr:,} val rows")

# Search space — Ray Tune samples from these distributions
param_space = {
    "params": {
        "objective":        "reg:squarederror",
        "eval_metric":      ["rmse"],
        "tree_method":      "hist",
        "seed":             42,
        "max_depth":        tune.randint(3, 10),
        "eta":              tune.loguniform(0.01, 0.3),
        "subsample":        tune.uniform(0.6, 1.0),
        "colsample_bytree": tune.uniform(0.5, 1.0),
        "min_child_weight": tune.randint(1, 20),
    },
    "label_column": TARGET,
    "num_boost_round": 100,
    "datasets": {
        "train": tune_train_ds,
        "valid": tune_val_ds,
    },
    "scaling_config": ScalingConfig(num_workers=2, use_gpu=False),
}

# ASHA Scheduler — aggressive early stopping
scheduler = ASHAScheduler(
    time_attr="training_iteration",
    max_t=100,           # max boosting rounds per trial
    grace_period=20,     # minimum rounds before stopping a trial
    reduction_factor=2,
)

print("Hyperparameter search space defined.")
print("Starting Ray Tune search (4 trials × 100 rounds max)...")
print("(~5–10 min on t3.large)")

In [ ]:
tuner = tune.Tuner(
    XGBoostTrainer,
    param_space=param_space,
    tune_config=tune.TuneConfig(
        metric="valid-rmse",
        mode="min",
        scheduler=scheduler,
        num_samples=4,       # number of trials (increase on bigger instances)
    ),
    run_config=RunConfig(
        name="xgb_fare_tune",
        storage_path=os.path.expanduser("~/ray_results"),
    ),
)

t0 = time.time()
tune_results = tuner.fit()
tune_time = time.time() - t0

print(f"\nTuning complete in {tune_time:.0f}s")

# Show all trial results
results_df = tune_results.get_dataframe()
cols_to_show = ["valid-rmse",
                "config/params/max_depth", "config/params/eta",
                "config/params/subsample", "config/params/colsample_bytree",
                "config/params/min_child_weight"]
available_cols = [c for c in cols_to_show if c in results_df.columns]
print("\nTrial results (sorted by val RMSE):")
print(results_df[available_cols].sort_values("valid-rmse").round(4).to_string())

In [ ]:
# Retrieve the best trial
best_result = tune_results.get_best_result(metric="valid-rmse", mode="min")
best_params = best_result.config["params"]

print("Best hyperparameters found:")
for k, v in best_params.items():
    if k not in ["objective", "eval_metric", "tree_method", "seed"]:
        print(f"  {k}: {v}")
print(f"  → val RMSE: {best_result.metrics.get('valid-rmse', 'N/A'):.4f}")

In [ ]:
# Retrain on the full training set with the best hyperparameters
print("Retraining with best params on full training set...")

final_trainer = XGBoostTrainer(
    params=best_params,
    label_column=TARGET,
    num_boost_round=200,    # more rounds now that params are tuned
    datasets={
        "train": train_ray_ds,
        "valid": val_ray_ds,
    },
    scaling_config=ScalingConfig(num_workers=2, use_gpu=False),
    run_config=RunConfig(
        name="xgb_fare_final",
        storage_path=os.path.expanduser("~/ray_results"),
        checkpoint_config=CheckpointConfig(num_to_keep=1),
    ),
)

t0 = time.time()
final_result = final_trainer.fit()
print(f"Final training complete in {time.time()-t0:.0f}s")

# Evaluate the tuned model
final_predictor = XGBoostPredictor.from_checkpoint(final_result.checkpoint)
final_preds_ds  = final_predictor.predict(test_ray_ds.drop_columns([TARGET]))
final_preds_df  = final_preds_ds.to_pandas()

y_pred_tuned = final_preds_df["predictions"].values

mae_tuned  = mean_absolute_error(y_test_vals, y_pred_tuned)
rmse_tuned = mean_squared_error(y_test_vals, y_pred_tuned, squared=False)
r2_tuned   = r2_score(y_test_vals, y_pred_tuned)

print(f"\nTuned model — test-set metrics:")
print(f"  MAE:  ${mae_tuned:.2f}  (baseline: ${mae_b:.2f})")
print(f"  RMSE: ${rmse_tuned:.2f}  (baseline: ${rmse_b:.2f})")
print(f"  R²:   {r2_tuned:.4f}  (baseline: {r2_b:.4f})")

---
## Section 7 — Model Evaluation

Three complementary views of model quality:
1. **Actual vs. Predicted** — how well-calibrated is the model?
2. **Residual analysis** — are errors systematic (suggesting missing features)?
3. **Feature importance** — which features drive the predictions most?

In [ ]:
# Use a 10k subsample for plotting clarity
plot_idx   = np.random.default_rng(0).integers(0, len(y_test_vals), 10_000)
y_actual   = y_test_vals[plot_idx]
y_pred_plot = y_pred_tuned[plot_idx]
residuals  = y_actual - y_pred_plot

fig, axes = plt.subplots(1, 3, figsize=(16, 5))
fig.suptitle("Tuned XGBoost — Fare Prediction Evaluation", fontsize=13)

# --- Panel 1: Actual vs. Predicted ---
ax = axes[0]
lim = (0, 80)
ax.hexbin(y_actual, y_pred_plot, gridsize=50, cmap="viridis",
          extent=(*lim, *lim), mincnt=1)
ax.plot(lim, lim, "r--", linewidth=1.5, label="Perfect prediction")
ax.set_xlabel("Actual Fare ($)")
ax.set_ylabel("Predicted Fare ($)")
ax.set_title(f"Actual vs. Predicted\nR² = {r2_tuned:.4f}")
ax.set_xlim(lim)
ax.set_ylim(lim)
ax.legend(fontsize=9)

# --- Panel 2: Residuals vs. Predicted ---
ax = axes[1]
ax.hexbin(y_pred_plot, residuals, gridsize=50, cmap="RdYlBu",
          extent=(0, 80, -30, 30), mincnt=1)
ax.axhline(0, color="black", linewidth=1.5, linestyle="--")
ax.axhline(mae_tuned, color="red", linewidth=1, linestyle=":",
           label=f"+MAE = ${mae_tuned:.2f}")
ax.axhline(-mae_tuned, color="red", linewidth=1, linestyle=":")
ax.set_xlabel("Predicted Fare ($)")
ax.set_ylabel("Residual (Actual − Predicted)")
ax.set_title(f"Residuals\nMAE = ${mae_tuned:.2f}, RMSE = ${rmse_tuned:.2f}")
ax.set_xlim(0, 80)
ax.set_ylim(-30, 30)
ax.legend(fontsize=9)

# --- Panel 3: Residual distribution ---
ax = axes[2]
ax.hist(residuals.clip(-25, 25), bins=60, color="steelblue",
        alpha=0.8, edgecolor="none", density=True)
ax.axvline(0, color="black", linewidth=1.5, linestyle="--")
ax.axvline(np.mean(residuals), color="red", linewidth=1.5,
           label=f"Mean error: ${np.mean(residuals):.2f}")
ax.set_xlabel("Residual ($)")
ax.set_ylabel("Density")
ax.set_title("Residual Distribution")
ax.legend(fontsize=9)

plt.tight_layout()
plt.savefig("model_evaluation.png", dpi=120, bbox_inches="tight")
plt.show()
print("Saved: model_evaluation.png")

In [ ]:
# Load the underlying xgb.Booster from the checkpoint for feature importance
# (XGBoostPredictor wraps it; we can get it with get_model())
booster = final_predictor.get_model()

# Feature importance — three views
importance_types = ["weight", "gain", "cover"]
importance_labels = {
    "weight": "Frequency\n(# times used in splits)",
    "gain":   "Gain\n(avg improvement per split)",
    "cover":  "Cover\n(avg # samples per split)",
}

fig, axes = plt.subplots(1, 3, figsize=(16, 5))
fig.suptitle("XGBoost Feature Importance — NYC Taxi Fare Prediction", fontsize=13)

for ax, imp_type in zip(axes, importance_types):
    scores = booster.get_score(importance_type=imp_type)
    # Map feature names (f0, f1...) back to column names
    feat_names = FEATURE_COLS_MODEL
    named_scores = {}
    for k, v in scores.items():
        # XGBoost uses 'f0', 'f1' etc. when trained via Ray Train
        idx_str = k.replace("f", "")
        if idx_str.isdigit() and int(idx_str) < len(feat_names):
            named_scores[feat_names[int(idx_str)]] = v
        else:
            named_scores[k] = v

    if named_scores:
        sorted_scores = sorted(named_scores.items(), key=lambda x: x[1])
        features, vals = zip(*sorted_scores)
        ax.barh(features, vals, color="steelblue", alpha=0.8)
        ax.set_title(f"{imp_type.capitalize()} Importance")
        ax.set_xlabel(importance_labels[imp_type])
        ax.grid(axis="x", alpha=0.3)
    else:
        ax.set_title(f"{imp_type.capitalize()} (unavailable)")

plt.tight_layout()
plt.savefig("feature_importance.png", dpi=120, bbox_inches="tight")
plt.show()
print("Saved: feature_importance.png")

In [ ]:
# Error breakdown by rate code — are we worse for special fares?
test_eval_df = test_split.copy()
test_eval_df["predicted_fare"] = y_pred_tuned
test_eval_df["abs_error"]      = np.abs(y_pred_tuned - y_test_vals)

rc_labels = {1: "Standard", 2: "JFK", 3: "Newark", 4: "Nassau",
             5: "Negotiated", 6: "Group"}

rc_errors = (
    test_eval_df
    .groupby("RatecodeID")
    .agg(
        n=("abs_error", "count"),
        mae=("abs_error", "mean"),
        avg_fare=("fare_amount", "mean"),
    )
    .round(2)
)
rc_errors["label"] = rc_errors.index.map(rc_labels)
rc_errors["pct_error"] = (rc_errors["mae"] / rc_errors["avg_fare"] * 100).round(1)

print("MAE by Rate Code:")
print(rc_errors[["label", "n", "avg_fare", "mae", "pct_error"]].to_string())
print()
print("Interpretation:")
print("  • RatecodeID=2 (JFK flat rate ~$70) should have very low MAE")
print("  • RatecodeID=5 (Negotiated) may be higher — those fares are unpredictable")
print("  • High pct_error for small rate codes (3, 4, 6) is expected with few samples")

---
## Section 8 — Saving the Model to S3 & Batch Prediction

### Workflow

```
Ray Train checkpoint  →  extract xgb.Booster  →  pickle  →  s3://YOUR_BUCKET/models/
                                                             ↓
New trip data  →  Ray Data map_batches(predict_batch)  →  predictions Parquet on S3
```

In production you'd use **Ray Serve** to wrap this into a REST endpoint, but for this tutorial we demonstrate offline batch prediction — a common pattern for nightly fare estimation or analytics.

In [ ]:
# --- Save the booster to S3 ---
model_bytes = pickle.dumps(booster)

s3_client = boto3.client("s3")
model_key  = "ray-tutorial/models/xgb_fare_model.pkl"

s3_client.put_object(
    Bucket=YOUR_BUCKET_NAME,
    Key=model_key,
    Body=model_bytes,
    ContentType="application/octet-stream",
)
print(f"Model saved to s3://{YOUR_BUCKET_NAME}/{model_key}")
print(f"Model size: {len(model_bytes) / 1024:.1f} KB")

# Also save in XGBoost's native format (JSON — more portable)
booster.save_model("/tmp/xgb_fare_model.json")
with open("/tmp/xgb_fare_model.json", "rb") as f:
    s3_client.put_object(
        Bucket=YOUR_BUCKET_NAME,
        Key="ray-tutorial/models/xgb_fare_model.json",
        Body=f.read(),
        ContentType="application/json",
    )
print(f"Native JSON model saved to s3://{YOUR_BUCKET_NAME}/ray-tutorial/models/xgb_fare_model.json")

In [ ]:
# --- Batch prediction with Ray Data ---
# Simulate "new" trips — April 2023 data
print("Loading April 2023 data for batch prediction...")

APRIL_PATH = "nyc-tlc/trip data/yellow_tripdata_2023-04.parquet"
april_raw  = ray.data.read_parquet(APRIL_PATH, filesystem=s3_anon)
april_ds   = april_raw.map_batches(prepare_for_model, batch_format="pandas")

# Sample 100k rows for a fast demo
april_sample_ds = april_ds.limit(100_000)
print(f"April sample dataset ready.")

In [ ]:
# Batch-predict using map_batches — each worker loads the model once
# We use ray.put() to share the model bytes across workers (Object Store pattern!)
model_bytes_ref = ray.put(model_bytes)

def predict_batch(df: pd.DataFrame, model_ref) -> pd.DataFrame:
    """Load model from Object Store and predict fares for a batch."""
    import pickle
    import xgboost as xgb

    booster = pickle.loads(model_ref)
    features = [c for c in FEATURE_COLS_MODEL if c in df.columns]
    dmatrix = xgb.DMatrix(df[features])
    df["predicted_fare"] = booster.predict(dmatrix).round(2)
    return df


print("Running batch predictions on April 2023 data...")
t0 = time.time()

# Drop the target column (it's in our dataset; in production it wouldn't exist)
features_only_ds = april_sample_ds.drop_columns([TARGET])

predictions_ds = features_only_ds.map_batches(
    predict_batch,
    batch_format="pandas",
    fn_kwargs={"model_ref": model_bytes_ref},
)

# Write predictions to S3
preds_s3_path = f"s3://{YOUR_BUCKET_NAME}/ray-tutorial/results/april_predictions/"
predictions_ds.write_parquet(preds_s3_path)

print(f"Batch predictions written to {preds_s3_path}")
print(f"Elapsed: {time.time()-t0:.1f}s")

# Show a sample
sample_preds = predictions_ds.limit(10).to_pandas()
print("\nSample predictions:")
print(sample_preds[["trip_distance", "RatecodeID", "pickup_hour",
                     "predicted_fare"]].to_string())

In [ ]:
# Compare predicted vs. actual for April (we do have the actual fares)
april_with_actual_ds = april_sample_ds
april_preds_ds = april_with_actual_ds.map_batches(
    predict_batch,
    batch_format="pandas",
    fn_kwargs={"model_ref": model_bytes_ref},
)
april_eval_df = april_preds_ds.to_pandas()

mae_april  = mean_absolute_error(april_eval_df[TARGET], april_eval_df["predicted_fare"])
rmse_april = mean_squared_error(april_eval_df[TARGET], april_eval_df["predicted_fare"],
                                squared=False)
r2_april   = r2_score(april_eval_df[TARGET], april_eval_df["predicted_fare"])

print("Out-of-time evaluation — April 2023 (model trained on Jan–Mar):")
print(f"  MAE:  ${mae_april:.2f}")
print(f"  RMSE: ${rmse_april:.2f}")
print(f"  R²:   {r2_april:.4f}")
print()
print("If April metrics are similar to the test set → no data drift.")
print("If significantly worse → fare structure may have changed (e.g. rate hike).")

---
## Section 9 — Cleanup & Summary

In [ ]:
ray.shutdown()
print("Ray shut down.")

# Summary table
print("\n" + "=" * 62)
print("MODEL PERFORMANCE SUMMARY")
print("=" * 62)
print(f"{'Model':<35} {'MAE ($)':>8} {'RMSE ($)':>9} {'R²':>7}")
print("-" * 62)
print(f"{'Serial XGBoost (500k sample)':<35} {mae_b:>8.2f} {rmse_b:>9.2f} {r2_b:>7.4f}")
print(f"{'Ray Train XGBoost (full, default params)':<35} {mae_ray:>8.2f} {rmse_ray:>9.2f} {r2_ray:>7.4f}")
print(f"{'Ray Train XGBoost (tuned)':<35} {mae_tuned:>8.2f} {rmse_tuned:>9.2f} {r2_tuned:>7.4f}")
print(f"{'April 2023 (out-of-time)':<35} {mae_april:>8.2f} {rmse_april:>9.2f} {r2_april:>7.4f}")
print("=" * 62)

# List S3 outputs
print(f"\nFiles written to s3://{YOUR_BUCKET_NAME}/ray-tutorial/")
try:
    s3_client = boto3.client("s3")
    resp = s3_client.list_objects_v2(Bucket=YOUR_BUCKET_NAME, Prefix="ray-tutorial/models/")
    for obj in resp.get("Contents", []):
        print(f"  {obj['Key']:60s} {obj['Size']/1024:8.1f} KB")
    resp2 = s3_client.list_objects_v2(
        Bucket=YOUR_BUCKET_NAME, Prefix="ray-tutorial/results/april_predictions/"
    )
    for obj in resp2.get("Contents", [])[:5]:
        print(f"  {obj['Key']:60s} {obj['Size']/1024:8.1f} KB")
except Exception as e:
    print(f"  Could not list: {e}")

# Local files
print("\nLocal files created:")
for f in ["fare_eda.png", "model_evaluation.png", "feature_importance.png"]:
    if os.path.exists(f):
        print(f"  {f:40s} {os.path.getsize(f)/1024:.1f} KB")

## Summary

### What you built

| Step | Tool | What happened |
|------|------|---------------|
| Data loading | `ray.data.read_parquet` | Read 3 months (~9M rows) from S3 in parallel |
| Cleaning & features | `map_batches` | Parallel pandas transforms across all partitions |
| Baseline model | `xgb.XGBRegressor` | Serial XGBoost on 500k sample |
| Distributed training | `XGBoostTrainer` + `ScalingConfig` | Ray spawns workers, data shards automatically |
| Hyperparameter tuning | `ray.tune.Tuner` + `ASHAScheduler` | 4 trials with early stopping |
| Evaluation | sklearn metrics + matplotlib | MAE, RMSE, R², feature importance |
| Model persistence | `boto3.put_object` | Saved to S3 as `.pkl` and `.json` |
| Batch prediction | `map_batches` + `ray.put()` | Model in Object Store → parallel predict → Parquet on S3 |

### Interpretation

A MAE of ~$2–3 means the model is off by about **1 stop's worth** of metered fare on average. The key drivers in order are:

1. **`RatecodeID`** — flat-rate codes (JFK=$70, Newark) dominate once learned
2. **`trip_distance`** — $0.50 per 1/5 mile in the meter
3. **`trip_duration_min`** — slow traffic charges
4. **`PULocationID` / `DOLocationID`** — zone-level patterns (airports, bridges)
5. **`pickup_hour`** — very mild effect; TLC fare formula doesn't have a surge multiplier

### Next steps

- **Ray Serve** — wrap the XGBoost model in a REST API endpoint (real-time predictions)
- **More features** — weather data (open-meteo API), events calendar, holiday flags
- **Multi-month training** — add 2022 data; compare seasonal drift
- **SHAP values** — `shap` library + XGBoost for per-prediction explanation
- [Ray Train docs](https://docs.ray.io/en/latest/train/train.html) | [Ray Tune docs](https://docs.ray.io/en/latest/tune/index.html) | [XGBoost-Ray](https://docs.ray.io/en/latest/train/examples/xgboost/xgboost-ray-example.html)